<a href="https://colab.research.google.com/github/jetsonmom/test/blob/main/mnt_zjpg_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ultralytics
from ultralytics import YOLO

In [ ]:
import os
import numpy as np
import random
import shutil
!pip install natsort

from tqdm import tqdm
from natsort import natsorted

def listdir(path):
    return natsorted(os.listdir(path))

def makedirs(path):
    os.makedirs(path, exist_ok = True)

# path 설정
import os

directory_path = '/home/ubuntu/5. AffectNet/'

# 디렉토리가 존재하지 않으면 생성합니다.
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
path = '/home/ubuntu/5. AffectNet/'


In [ ]:
# 메인 폴더 생성
for mode in ['train_set', 'val_set']:
    for class_no in range(8):
        os.makedirs(f'{path}/new_dataset/{mode}/{class_no}', exist_ok = True)

os.makedirs(f'{path}/new_dataset', exist_ok = True)

# 데이터 분류
for mode in ['train_set', 'val_set']:
    img_list = os.listdir(f'{path}/{mode}/z.jpg')
    for cnt, img_name in enumerate(tqdm(img_list)):
        try:
            # if cnt == 100: break
            label_name = f'{img_name.split(".")[0]}_exp.npy'
            class_no = np.load(f'{path}/{mode}/annotations/{label_name}')
            # 이미지 복사
            from_path = f'{path}/{mode}/images/{img_name}'
            to_path = f'{path}/new_dataset/{mode}/{class_no}/{img_name}'
            shutil.copy(from_path, to_path)
        except:
            print(f'에러 발생: {img_name}')
            continue


In [ ]:
import os
import numpy as np
import shutil
from tqdm import tqdm

def makedirs(path):
    os.makedirs(path, exist_ok=True)

# 메인 데이터셋 디렉터리 설정
directory_path = '/home/ubuntu/5. AffectNet/'

# 메인 디렉터리가 존재하지 않는다면 생성
makedirs(directory_path)

# 새 데이터셋을 위한 경로 설정
path = directory_path  # 메인 디렉터리 사용
new_dataset_path = os.path.join(path, 'new_dataset')
makedirs(new_dataset_path)

# 훈련 세트와 검증 세트를 위한 디렉터리와 클래스 서브디렉터리 생성
for mode in ['train_set', 'val_set']:
    for class_no in range(8):
        class_path = os.path.join(new_dataset_path, mode, str(class_no))
        makedirs(class_path)

# 이미지를 클래스에 따라 분류하는 함수
def sort_images(mode, images_path, annotations_path):
    img_list = os.listdir(images_path)
    for img_name in tqdm(img_list):
        try:
            # 레이블 추출
            label_name = f'{img_name.split(".")[0]}_exp.npy'
            class_no = np.load(os.path.join(annotations_path, label_name))
            # 이미지를 새 위치로 복사
            from_path = os.path.join(images_path, img_name)
            to_path = os.path.join(new_dataset_path, mode, str(class_no), img_name)
            shutil.copy(from_path, to_path)
        except Exception as e:
            print(f'{img_name} 에러 발생: {e}')

# 훈련 세트와 검증 세트에 대한 분류 과정 실행
for mode in ['train_set', 'val_set']:
    images_path = os.path.join(path, mode, 'images')
    annotations_path = os.path.join(path, mode, 'annotations')
    sort_images(mode, images_path, annotations_path)


In [ ]:
import cv2
import matplotlib.pyplot as plt

# OpenCV Haar Cascade를 로드합니다.
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 이미지를 로드합니다.
image = cv2.imread('/content/z.jpg')

# 이미지를 그레이스케일로 변환합니다.
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 얼굴을 감지합니다.
faces = face_cascade.detectMultiScale(gray_image, 1.1, 4)

# 감지된 얼굴을 추출합니다.
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
    face = image[y:y+h, x:x+w]
    plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
    plt.show()

# 전체 이미지에 그려진 사각형을 확인합니다.
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
!pip install mtcnn

from mtcnn import MTCNN
import cv2
import os

detector = MTCNN()

# 단일 이미지 파일 경로 설정
image_path = '/home/ubuntu/z.jpg'

# 추출된 얼굴을 저장할 경로 설정
output_path = '/home/ubuntu/extracted_faces/'

# 저장할 경로가 없으면 생성
if not os.path.exists(output_path):
    os.makedirs(output_path)

# 이미지 로드
image = cv2.imread(image_path)

# 이미지가 제대로 로드되었는지 확인
if image is not None:
    # 얼굴 감지
    faces = detector.detect_faces(image)

    # 감지된 얼굴 처리
    for i, face in enumerate(faces):
        x, y, width, height = face['box']
        cropped_face = image[y:y+height, x:x+width]

        # 추출한 얼굴 이미지 저장
        face_file_name = f"z_face_{i}.jpg"
        face_file_path = os.path.join(output_path, face_file_name)
        cv2.imwrite(face_file_path, cropped_face)
        print(f"Extracted face {i} saved to {face_file_path}")
else:
    print('Image not loaded correctly.')



In [ ]:
from mtcnn import MTCNN
import cv2
from matplotlib import pyplot as plt

# MTCNN detector 초기화
detector = MTCNN()

# 이미지 파일 경로 설정
image_path = '/home/ubuntu/z.jpg'  # 로컬 경로
output_path = '/content/extracted_faces/'  # Colab 경로

# Colab에서 이미지를 표시하기 위한 함수
def show_image(image, title=''):
    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# 이미지 로드
image = cv2.imread(image_path)

# 이미지가 제대로 로드되었는지 확인
if image is not None:
    # 얼굴 감지
    faces = detector.detect_faces(image)

    if faces:
        for i, face in enumerate(faces):
            x, y, width, height = face['box']
            cropped_face = image[y:y+height, x:x+width]

            # 추출된 얼굴 이미지를 저장하고 보여주기
            face_file_name = f"z_face_{i}.jpg"
            face_file_path = os.path.join(output_path, face_file_name)
            cv2.imwrite(face_file_path, cropped_face)
            show_image(cropped_face, f"Extracted Face {i}")
            print(f"Extracted face {i} saved to {face_file_path}")
    else:
        print('No faces detected.')
else:
    print('Image not loaded correctly.')


In [ ]:
from mtcnn import MTCNN
import cv2
from matplotlib import pyplot as plt

# MTCNN detector 초기화
detector = MTCNN()

# 이미지 파일 경로 설정
image_path = '/home/ubuntu/z.jpg'  # 로컬 경로
output_path = '/content/extracted_faces/'  # Colab 경로

# Colab에서 이미지를 표시하기 위한 함수
def show_image(image, title=''):
    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# 이미지 로드
image = cv2.imread(image_path)

# 이미지가 제대로 로드되었는지 확인
if image is not None:
    # 얼굴 감지
    faces = detector.detect_faces(image)

    if faces:
        for i, face in enumerate(faces):
            x, y, width, height = face['box']
            cropped_face = image[y:y+height, x:x+width]

            # 추출된 얼굴 이미지를 저장하고 보여주기
            face_file_name = f"z_face_{i}.jpg"
            face_file_path = os.path.join(output_path, face_file_name)
            cv2.imwrite(face_file_path, cropped_face)
            show_image(cropped_face, f"Extracted Face {i}")
            print(f"Extracted face {i} saved to {face_file_path}")
    else:
        print('No faces detected.')
else:
    print('Image not loaded correctly.')


In [ ]:
from mtcnn import MTCNN
import cv2
from matplotlib import pyplot as plt

# MTCNN detector 초기화
detector = MTCNN()

# 이미지 파일 경로 설정
image_path = '/content/z.jpg'  # Colab에 업로드된 파일 경로

# 이미지 로드
image = cv2.imread(image_path)

# Colab에서 이미지와 얼굴을 표시하기 위한 함수
def show_image(image, faces=None, title=''):
    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    if faces is not None:
        for face in faces:
            x, y, width, height = face['box']
            rect = plt.Rectangle((x, y), width, height, fill=False, color='red')
            plt.gca().add_patch(rect)
    plt.show()

# 이미지가 제대로 로드되었는지 확인
if image is not None:
    # 얼굴 감지
    faces = detector.detect_faces(image)

    # 감지된 얼굴에 경계 상자를 그린 후 전체 이미지 표시
    show_image(image, faces, 'Detected Faces')

    # 감지된 각 얼굴에 대한 반복문
    for i, face in enumerate(faces):
        x, y, width, height = face['box']
        cropped_face = image[y:y+height, x:x+width]

        # 개별적으로 감지된 얼굴 표시
        plt.figure(figsize=(3, 3))
        plt.imshow(cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB))
        plt.title(f"Face {i}")
        plt.axis('off')
        plt.show()
else:
    print('Image not loaded correctly.')


In [ ]:
from mtcnn import MTCNN
import cv2
from matplotlib import pyplot as plt

# MTCNN detector 초기화
detector = MTCNN()

# 이미지 로드
image_path = '/mnt/z.jpg'
image = cv2.imread(image_path)

# Colab에서 이미지와 얼굴을 표시하기 위한 함수
def show_image(image, faces=None, title=''):
    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    if faces is not None:
        for face in faces:
            x, y, width, height = face['box']
            rect = plt.Rectangle((x, y), width, height, fill=False, color='red')
            plt.gca().add_patch(rect)
    plt.show()

# 이미지가 제대로 로드되었는지 확인
if image is not None:
    # 얼굴 감지
    faces = detector.detect_faces(image)

    # 감지된 얼굴에 경계 상자를 그린 후 전체 이미지 표시
    show_image(image, faces, 'Detected Faces')
else:
    print('Image not loaded correctly.')
